In [2]:
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

def resample_to_dem(era5_path, dem_path, out_path):
    # Open DEM as reference (30m, EPSG:32644)
    with rasterio.open(dem_path) as dem_src:
        dst_transform = dem_src.transform
        dst_crs = dem_src.crs
        dst_height = dem_src.height
        dst_width = dem_src.width

    # Open ERA5
    with rasterio.open(era5_path) as src:
        src_data = src.read(1)  # read first band
        src_transform = src.transform
        src_crs = src.crs

        # Output array
        dst_data = np.empty((dst_height, dst_width), dtype=np.float32)

        # Reproject to DEM grid
        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear  # or nearest for categorical
        )

    # Save reprojected raster
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=dst_height,
        width=dst_width,
        count=1,
        dtype=dst_data.dtype,
        crs=dst_crs,
        transform=dst_transform,
    ) as dst:
        dst.write(dst_data, 1)

# Example usage:
era5_file = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif"
dem_file  = r"C:\Users\Ankit\Datasets_Forest_fire\merged_DEM_30m_32644_aligned_filled.tif"
out_file  = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_t2m_resampled_30m.tif"

resample_to_dem(era5_file, dem_file, out_file)
